In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
count = application_df.APPLICATION_TYPE.value_counts()
count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application = list(count[count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
classification_count = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1250].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

c:\Users\nuhkh\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(labels=application_cat, axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=18)

c:\Users\nuhkh\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Attempt One At Improved Accuracy - Adding additonal hidden layer

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_number = len(X_train[0])
hidden_layer1 = 100
hidden_layer2 = 60
hidden_layer3 = 45

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, activation='relu', input_dim = input_number))

# Second and Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4400      
                                                                 
 dense_5 (Dense)             (None, 60)                6060      
                                                                 
 dense_6 (Dense)             (None, 45)                2745      
                                                                 
 dense_7 (Dense)             (None, 1)                 46        
                                                                 
Total params: 13,251
Trainable params: 13,251
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [18]:
# Implement a callback to store the model's weights every 5 epochs
from tensorflow.keras.callbacks import ModelCheckpoint
import os

os.makedirs("checkpoints/",exist_ok=True)

cp_callback = ModelCheckpoint(filepath="checkpoints/weights.{epoch:02d}.hdf5", verbose=1, save_weights_only=True, save_freq=5 * 804)

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5665 - accuracy: 0.7278
Epoch 2/100
804/804 [==============================] - 1s 991us/step - loss: 0.5528 - accuracy: 0.7311
Epoch 3/100
804/804 [==============================] - 1s 966us/step - loss: 0.5505 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 1s 992us/step - loss: 0.5484 - accuracy: 0.7337
Epoch 5/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5480 - accuracy: 0.7347
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7352
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7350
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 1s 998us/step - loss: 0.5449 - accuracy: 0.7353
Epoch 9/100
804/804 [=========================

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5857 - accuracy: 0.7231 - 256ms/epoch - 955us/step
Loss: 0.5856528878211975, Accuracy: 0.7231487035751343


Attempt two At Improved Accuracy - Adjusting neuron and epoch count with additonal hidden layer

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_number = len(X_train[0])
hidden_layer1 = 120
hidden_layer2 = 70
hidden_layer3 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer1, activation='relu', input_dim = input_number))

# Second and Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))
nn2.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))


# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 120)               5280      
                                                                 
 dense_13 (Dense)            (None, 70)                8470      
                                                                 
 dense_14 (Dense)            (None, 50)                3550      
                                                                 
 dense_15 (Dense)            (None, 1)                 51        
                                                                 
Total params: 17,351
Trainable params: 17,351
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [28]:
# Implement a callback to store the model's weights every 5 epochs
from tensorflow.keras.callbacks import ModelCheckpoint
import os

os.makedirs("checkpoints/",exist_ok=True)

cp_callback = ModelCheckpoint(filepath="checkpoints/weights.{epoch:02d}.hdf5", verbose=1, save_weights_only=True, save_freq=5 * 804)

In [29]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=150, callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 976us/step - loss: 0.5658 - accuracy: 0.7249
Epoch 2/150
804/804 [==============================] - 1s 988us/step - loss: 0.5529 - accuracy: 0.7324
Epoch 3/150
804/804 [==============================] - 1s 966us/step - loss: 0.5505 - accuracy: 0.7315
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7335
Epoch 5/150
766/804 [===========================>..] - ETA: 0s - loss: 0.5465 - accuracy: 0.7347
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7339
Epoch 6/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7344
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5454 - accuracy: 0.7352
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7354
Epoch 9/150
804/804 [===========================

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5931 - accuracy: 0.7233 - 303ms/epoch - 1ms/step
Loss: 0.5931432843208313, Accuracy: 0.7232652902603149


Attempt Three At Improved Accuracy - Changing activation function and a fourth hidden layer

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_number = len(X_train[0])
hidden_layer1 = 120
hidden_layer2 = 70
hidden_layer3 = 50
hidden_layer4 = 34

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer1, activation='sigmoid', input_dim = input_number))

# Second, Third, and Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))
nn3.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))
nn3.add(tf.keras.layers.Dense(units=hidden_layer4, activation='relu'))


# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 120)               5280      
                                                                 
 dense_22 (Dense)            (None, 70)                8470      
                                                                 
 dense_23 (Dense)            (None, 50)                3550      
                                                                 
 dense_24 (Dense)            (None, 34)                1734      
                                                                 
 dense_25 (Dense)            (None, 1)                 35        
                                                                 
Total params: 19,069
Trainable params: 19,069
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn3.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [35]:
# Implement a callback to store the model's weights every 5 epochs
from tensorflow.keras.callbacks import ModelCheckpoint
import os

os.makedirs("checkpoints/",exist_ok=True)

cp_callback = ModelCheckpoint(filepath="checkpoints/weights.{epoch:02d}.hdf5", verbose=1, save_weights_only=True, save_freq=5 * 804)

In [36]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5787 - accuracy: 0.7158
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5587 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5565 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7316
Epoch 5/100
779/804 [============================>.] - ETA: 0s - loss: 0.5518 - accuracy: 0.7338
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7316
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5505 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7325
Epoch 9/100
804/804 [==============================] -

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5753 - accuracy: 0.7216 - 255ms/epoch - 953us/step
Loss: 0.575346052646637, Accuracy: 0.7216326594352722


In [38]:
 # Export to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")